In [1]:
from dataclasses import dataclass, field

from numpy.random import randint

In [2]:
switch_dict = {
            "up": "down",
            "down": "up",
        }
floor_lims = (4, 12)
people_lim = 4

nfloors = lambda *args: randint(*args)
hasppl = lambda: bool(randint(2))
nppl = lambda y: randint(y)

def rand_floor(blg_floors: int):
    if (hasppl() or hasppl()):
        return [randint(blg_floors) for x in range(nppl(people_lim))]
    else:
        return []
def randbld(
    blg_floors: int = nfloors(*floor_lims),
    ):
    return [rand_floor(blg_floors) for i in range(blg_floors)]
randbld()

[[1, 0, 4], [8, 7], [], [3, 9], [], [8], [], [10, 5, 2], [2], [], []]

In [3]:
# Floors:    G     1      2        3     4      5      6         Answers:
tests = [
         [ ( (),   (),    (5,5,5), (),   (2,2),    (),    () ),     [0, 2, 5, 0]          ],
         [ ( (),   (),    (1,1),   (),   (),    (),    () ),     [0, 2, 1, 0]          ],
         [ ( (),   (3,),  (4,),    (),   (5,),  (),    () ),     [0, 1, 2, 3, 4, 5, 0] ],
         [ ( (),   (0,),  (),      (),   (2,),  (3,),  () ),     [0, 5, 4, 3, 2, 1, 0] ]
         ]

In [16]:
@dataclass
class Person:
    want_floor: int = field()
    start_floor: int = field()
    cur_floor: int = None

    def __post_init__(self):
        self.cur_floor = self.start_floor

    @property
    def going_up(self):
        return self.want_floor > self.cur_floor
    
    @property
    def going_down(self):
        return self.want_floor < self.cur_floor

    @property
    def at_dest(self) -> bool:
        return self.want_floor == self.cur_floor

    def dir_is_aligned(self,
                       lift_dir: str,
                       ):
        down = (self.going_down and lift_dir == "down")
        up = (self.going_up and lift_dir == "up")
        return (up or down)
    
    def can_load(self,
                 lift_dir: str,
                 lift_at_cap: bool,
                 ):
        is_aligned = self.dir_is_aligned(lift_dir)
        return (is_aligned and not lift_at_cap)
    
    @property
    def can_unload(self):
        return self.at_dest

@dataclass
class People:
    lst: list[Person] = field(default_factory=list)
    cur_floor: int = None
    
    @property
    def n(self) -> int:
        return len(self.lst)

    def set_cur_floor(self, floor: int) -> None:
        for x in self.lst:
            x.cur_floor = floor
        self.cur_floor = floor
    
    @property
    def ppl_at_dest(self) -> list[Person]:
        return [x for x in self.lst if x.at_dest]

    @property
    def ppl_not_at_dest(self) -> list[Person]:
        return [x for x in self.lst if not x.at_dest]

    @property
    def n_at_dest(self) -> int:
        return len(self.ppl_at_dest)

    @property
    def n_not_at_dest(self) -> int:
        return self.n - self.n_at_dest
    
    def __repr__(self):
        return f"n {self.n} | atdest {self.n_at_dest} | natdest {self.n_not_at_dest}"

    @property
    def ppl_going_up(self) -> list[Person]:
        return [x for x in self.ppl_not_at_dest if x.going_up]
    
    @property
    def ppl_going_down(self) -> list[Person]:
        return [x for x in self.ppl_not_at_dest if x.going_down]
    
    @property
    def has_going_up(self) -> bool:
        return len(self.ppl_going_up) > 0

    @property
    def has_going_down(self) -> bool:
        return len(self.ppl_going_down) > 0

    @property
    def want_up_floors(self) -> list[int]:
        return [x.want_floor for x in self.lst if x.going_up]
    
    @property
    def want_down_floors(self) -> list[int]:
        return [x.want_floor for x in self.lst if x.going_down]
    
    @property
    def want_floors(self) -> list[int]:
        return self.want_up_floors + self.want_down_floors

    @property
    def cur_floors(self) -> list[int]:
        return [x.cur_floor for x in self.lst]

    @classmethod
    def from_floor(cls, cur_floor: int, want_floor_list: list[int]) -> object:
        wfl = list(want_floor_list)
        wfl.sort()
        ppl = [Person(x, cur_floor) for x in wfl]
        return cls(lst=ppl)

    def ppl_can_load(
            self,
            lift_dir: str,
            lift_at_cap: bool,
            ) -> list[Person]:
        return [m for m in self.ppl_not_at_dest if m.can_load(lift_dir, lift_at_cap)]
    
    def add_person(self, person: Person) -> None:
        self.lst.append(person)
    
    def add_ppl(self, ppl: list[Person]) -> None:
        self.lst = self.lst + ppl
    
    def rm_person(self, person_idx: int) -> Person:
        person = self.lst[person_idx]
        self.lst = [self.lst[i] for i in range(self.n) if i != person_idx]
        return person

    def get_next_load_idx(
            self,
            lift_dir: str,
            ) -> int:
        lift_going_up = lift_dir == "up"
        if (lift_going_up and self.has_going_up):
            next_wf = min(self.want_up_floors)
        elif lift_going_up:
            next_wf = max(self.want_down_floors)
        elif (not lift_going_up and self.has_going_down):
            next_wf = max(self.want_down_floors)
        elif not lift_going_up:
            next_wf = min(self.want_up_floors)
        else:
            raise ValueError("need valid want person")
        return min([i for i in range(self.n) if self.lst[i].want_floor == next_wf])

    def get_next_unload_idx(self):
        return min([i for i in range(self.n) if self.lst[i].at_dest])


@dataclass
class Floor(People):
    i: int = field(default=None)
    want_list: list[int] = field(default_factory=list, repr=False)

    def get_ppl_lst(self, want_list: list[int]) -> list[Person]:
        return [Person(wf, self.i) for wf in want_list]

    def __post_init__(self) -> None:
        self.cur_floor = self.i
        self.lst = self.get_ppl_lst(self.want_list)

    @property
    def needs_visit(self) -> bool:
        return self.n_not_at_dest > 0

    def get_load_person(
            self,
            lift_dir: str,
            ) -> Person:
        next_load_idx = self.get_next_load_idx(lift_dir)
        return self.rm_person(next_load_idx)
    
    @classmethod
    def from_want_list(cls, i: int, want_list: list[int]):
        return Floor(i=i, want_list=want_list)
    
    def __repr__(self):
        return f"Floor {self.i}, n = {self.n}, want_floors = {self.want_floors}"
    

@dataclass
class Lift(People):
    capacity: int = field(default=None)
    direction: str = field(default="up")
    record: list[int] = field(default_factory=list)
    cur_floor: int = None
    called_floors: list[int] = field(default_factory=list)

    @property
    def spaces(self):
        return self.capacity - self.n

    @property
    def at_cap(self):
        return self.spaces == 0
    
    @property
    def is_empty(self):
        return self.n == 0
    
    @property
    def going_up(self):
        return self.direction == "up"
    
    @property
    def going_down(self):
        return self.direction == "down"

    @property
    def dest_floors(self):
        wfs = self.want_floors
        called = self.called_floors
        return list(set(wfs + called))

    def switch_dir(self):
        self.direction = switch_dict[self.direction]
        # print(f"switching to {self.direction} on {self.cur_floor}")
    
    def get_unload_person(self):
        unload_idx = self.get_next_unload_idx()
        return self.rm_person(unload_idx)

    def set_lift_floor(self, floor: int):
        if floor != self.record[-1]:
            self.record.append(floor)
        self.set_cur_floor(floor)

    def __repr__(self):
        return f"Lift[cur={self.cur_floor}, n={self.n}, wants={self.want_floors}, ppl at = {self.cur_floors}]"

    def __post_init__(self):
        self.record.append(0)
        self.set_cur_floor(0)

    @property
    def nrecs(self):
        return len(self.record)

@dataclass
class Building:
    floors: list[Floor]
    lift: Lift
    
    @staticmethod
    def get_floors(queues: list[tuple[int]]):
        _range = range(len(queues))
        return [Floor.from_want_list(i, queues[i]) for i in _range]
    
    def set_floors(self, floors: list[tuple[int]]):
        self.floors = Building.get_floors(floors)

    @property
    def called_floors(self):
        cf = [i for i in self._range if self.floors[i].needs_visit]
        return cf

    def __post_init__(self):
        self.set_floors(self.floors)
        self.nfloors = len(self.floors)
        self._range = range(self.nfloors)
        self.lift.called_floors = self.called_floors
    @property
    def called_up_floors(self):
        cuf = [i for i in self.called_floors if self.floors[i].has_going_up]
        return cuf
    @property
    def all_up_floors(self):
        return list(set(self.called_up_floors + self.lift.want_up_floors))
    @property
    def called_down_floors(self):
        cdf = [i for i in self.called_floors if self.floors[i].has_going_down]
        return cdf
    @property
    def all_down_floors(self):
        return list(set(self.called_down_floors + self.lift.want_down_floors))
    @property
    def floors_above_lift(self) -> list[Floor]:
        return [x.i for x in self.floors if x.i > self.lift.cur_floor]
    @property
    def floors_below_lift(self) -> list[Floor]:
        return [x.i for x in self.floors if x.i < self.lift.cur_floor]
    @property
    def called_up_above(self):
        return [i for i in self.called_up_floors if i in self.floors_above_lift]
    @property
    def called_up_below(self):
        return [i for i in self.called_up_floors if i in self.floors_below_lift]
    @property
    def called_down_above(self):
        return [i for i in self.called_down_floors if i in self.floors_above_lift]
    @property
    def called_down_below(self):
        return [i for i in self.called_down_floors if i in self.floors_below_lift]
    @property
    def want_up_above(self):
        return [i for i in self.lift.want_up_floors if i in self.floors_above_lift]
    @property
    def want_up_below(self):
        return [i for i in self.lift.want_up_floors if i in self.floors_below_lift]
    @property
    def want_down_above(self):
        return [i for i in self.lift.want_down_floors if i in self.floors_above_lift]
    @property
    def want_down_below(self):
        return [i for i in self.lift.want_down_floors if i in self.floors_below_lift]
    @property
    def has_called_up_above(self):
        return len(self.called_up_above) > 0
    @property
    def has_called_up_below(self):
        return len(self.called_up_below) > 0
    @property
    def has_called_down_above(self):
        return len(self.called_down_above) > 0
    @property
    def has_called_down_below(self):
        return len(self.called_down_below) > 0
    @property
    def has_want_up_above(self):
        return len(self.want_up_above) > 0
    @property
    def has_want_up_below(self):
        return len(self.want_up_below) > 0
    @property
    def has_want_down_above(self):
        return len(self.want_down_above) > 0
    @property
    def has_want_down_below(self):
        return len(self.want_down_below) > 0
    @property
    def next_floor(self):
        if self.lift.going_up:
            if (self.has_called_up_above or self.has_want_up_above):
                return min(self.called_up_above + self.want_up_above)
            elif (self.has_called_down_above or self.has_want_down_above):
                return max(self.called_down_above + self.want_down_above)
            else:
                self.lift.switch_dir()
                return self.lift.cur_floor
        elif self.lift.going_down:
            if (self.has_called_down_below or self.has_want_down_below):
                return max(self.called_down_below + self.want_down_below)
            elif (self.has_called_up_below or self.has_want_up_below):
                return min(self.called_up_below + self.want_up_below)
            else:
                self.lift.switch_dir()
                return self.lift.cur_floor
        else:
            raise ValueError("lift should be going up or down")
    
    def set_next_floor(self):
        self.lift.set_lift_floor(self.next_floor)

    @property
    def n_called_up(self):
        return len(self.called_up_floors)
    @property
    def n_called_down(self) -> int:
        return len(self.called_down_floors)

    @property
    def has_called_up(self):
        return self.n_called_up > 0

    @property
    def has_called_down(self):
        return self.n_called_down > 0

    @property
    def stop_cond(self) -> bool:
        return (len(self.called_floors) == 0 and self.lift.is_empty)

    @property
    def lift_floor(self) -> Floor:
        return self.floors[self.lift.cur_floor]
    
    @property
    def can_load(self) -> bool:
        if self.lift.at_cap:
            return False
        else:
            up = (self.lift.going_up and self.lift_floor.has_going_up)
            down = (self.lift.going_down and self.lift_floor.has_going_down)
            return (up or down)
    
    @property
    def can_unload(self):
        return self.lift.n_at_dest > 0

    def load_one(self) -> None:
        if not self.lift.at_cap:
            load_person = self.lift_floor.get_load_person(
                self.lift.direction,
            )
        self.lift.add_person(load_person)

    def load(self) -> None:
        while self.can_load:
            self.load_one()

    def unload_one(self):
        unload_person = self.lift.get_unload_person()
        self.lift_floor.add_person(unload_person)

    def unload(self):
        while self.can_unload:
            self.unload_one()

    def turn(self):
        self.unload()
        self.load()
        self.set_next_floor()

    def run(self):
        while not self.stop_cond:
            self.turn()
        if self.lift.record[-1] != 0:
            self.lift.set_lift_floor(0)
        return self.lift.record
    
class Dinglemouse(object):
    def __init__(self, queues, capacity):
        self.bld = Building(queues,
                            Lift(capacity=capacity),
                            )     
    def theLift(self):
        return self.bld.run()

In [17]:
for i, test in enumerate(tests):
    queues, answer = test
    bld = Building(queues, Lift(capacity=5))
    res = bld.run()
    if res == answer:
        print(True)
    else:
        print(res, answer)

[0, 2, 5, 4, 2, 0] [0, 2, 5, 0]
True
True
True


In [18]:
bld.floors

[Floor 0, n = 1, want_floors = [],
 Floor 1, n = 0, want_floors = [],
 Floor 2, n = 1, want_floors = [],
 Floor 3, n = 1, want_floors = [],
 Floor 4, n = 0, want_floors = [],
 Floor 5, n = 0, want_floors = [],
 Floor 6, n = 0, want_floors = []]

In [35]:
bld = Building(randbld(), Lift(capacity=randint(1, 5)))
bld.run()
print(f"runs={bld.lift.nrecs}")
bld.floors

runs=23


[Floor 0, n = 1, want_floors = [],
 Floor 1, n = 4, want_floors = [],
 Floor 2, n = 0, want_floors = [],
 Floor 3, n = 2, want_floors = [],
 Floor 4, n = 0, want_floors = [],
 Floor 5, n = 0, want_floors = [],
 Floor 6, n = 1, want_floors = [],
 Floor 7, n = 0, want_floors = [],
 Floor 8, n = 2, want_floors = [],
 Floor 9, n = 1, want_floors = [],
 Floor 10, n = 0, want_floors = []]